In [1]:
# use pretty print library to get better output
import pprint

pp = pprint.PrettyPrinter()

In [2]:
# load train.txt
with open("dataset/train.txt") as f:
    # declare an empty list for later saving train data values
    train_value = []
    predict_value = []

    # read the train values line by line
    for line in f:
        # seperate every sequence number by comma
        sep_string = line.split(',')

        count = 0

        # using slice of list to get the value we acctually need
        while(count < 2):
            # first and second number sequence need the last three digits
            sep_string[count] = sep_string[count][-3:]
            count += 1
        
        # the third number sequence need last four digits and no '\n'
        sep_string[2] = sep_string[2][-5:-1]
        train_value.append(sep_string[:2])
        predict_value.append(sep_string[-1])

In [3]:
# show first five train value
pp.pprint(train_value[:5])

[['686', '617'], ['643', '401'], ['375', '414'], ['476', '627'], ['431', '387']]


In [4]:
# show first five predict value
pp.pprint(predict_value[:5])

['4983', '2824', '1601', '4023', '1638']


In [5]:
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

Using TensorFlow backend.


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense


model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)